In [93]:
import datetime
import numpy
import os
import pandas
import pathlib
import re
import requests
import sqlalchemy
import sqlalchemy.ext.declarative
import sqlalchemy.orm
import sqlalchemy.orm.decl_api
import time

# cfg

In [94]:
cfg = {
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "original.csv"
filename_george = "dados-george.csv"


def connect(cfg):
    list_hosts = ["localhost", "192.168.0.160"]
    for host in list_hosts:
        try:
            engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{cfg['user']}:{cfg['password']}@{host}:{cfg['port']}/{cfg['database']}", echo=True, pool_pre_ping=True)
            Session = sqlalchemy.orm.sessionmaker(bind=engine)
            Session.configure(bind=engine)
            session = Session()
            if engine.connect():
                return engine, session
        except Exception as e:
            print(f"problems with host {host}")


engine, session = connect(cfg)

problems with host localhost
2022-07-20 10:42:35,352 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-07-20 10:42:35,352 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-20 10:42:35,352 INFO sqlalchemy.engine.Engine select current_schema()
2022-07-20 10:42:35,352 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-07-20 10:42:35,361 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-07-20 10:42:35,361 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load data

In [95]:
dataframe = pandas.read_csv("original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

# "Tables"

In [96]:
Base = sqlalchemy.ext.declarative.declarative_base()

class DataSP(Base):
    __tablename__ = "data"

    seq = sqlalchemy.Column(sqlalchemy.BigInteger, primary_key=True)
    modified = sqlalchemy.Column(sqlalchemy.DateTime, nullable=True)
    institution_code = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    collection_code = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    catalog_number = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    basis_of_record = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    kingdom = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    phylum = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    classe = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    order = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    family = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    genus = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    specific_epithet = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    infraspecific_epithet = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    scientific_name = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    scientific_name_authorship = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    identified_by = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    year_identified = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    month_identified = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    day_identified = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    type_status = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    recorded_by = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    record_number = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    field_number = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    year = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    month = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    day = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    event_time = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    continent_ocean = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    country = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    state_province = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    county = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    locality = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    decimal_longitude = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    decimal_latitude = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    verbatim_longitude = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    verbatim_latitude = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    coordinate_precision = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    bounding_box = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    minimum_elevation_in_meters = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    maximum_elevation_in_meters = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    minimum_depth_in_meters = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    maximum_depth_in_meters = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    sex = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    preparation_type = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    individual_count = sqlalchemy.Column(sqlalchemy.BigInteger, nullable=True)
    previous_catalog_number = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    relationship_type = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    related_catalog_item = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    occurrence_remarks = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    barcode = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    imagecode = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    geo_flag = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    george = sqlalchemy.Column(sqlalchemy.Boolean, nullable=True)
    my_country = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    my_state = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    my_city = sqlalchemy.Column(sqlalchemy.String, nullable=True)

    def __repr__(self):
        return "DataSP(seq=%s, modified=%s, institution_code=%s, collection_code=%s, catalog_number=%s, " \
               "basis_of_record=%s, kingdom=%s, phylum=%s, classe=%s, order=%s, family=%s, genus=%s, " \
               "specific_epithet=%s, infraspecific_epithet=%s, scientific_name=%s, scientific_name_authorship=%s, " \
               "identified_by=%s, year_identified=%s, month_identified=%s, day_identified=%s, type_status=%s, " \
               "recorded_by=%s, record_number=%s, field_number=%s, year=%s, month=%s, day=%s, event_time=%s, " \
               "continent_ocean=%s, country=%s, state_province=%s, county=%s, locality=%s, decimal_longitude=%s, " \
               "decimal_latitude=%s, verbatim_longitude=%s, verbatim_latitude=%s, coordinate_precision=%s, " \
               "bounding_box=%s, minimum_elevation_in_meters=%s, maximum_elevation_in_meters=%s, " \
               "minimum_depth_in_meters=%s, maximum_depth_in_meters=%s, sex=%s, preparation_type=%s, " \
               "individual_count=%s, previous_catalog_number=%s, relationship_type=%s, related_catalog_item=%s, " \
               "occurrence_remarks=%s, barcode=%s, imagecode=%s, geo_flag=%s) "

# County is muncipio, condado
class County(Base):
    __tablename__ = "county"

    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    county = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    uf = sqlalchemy.Column(sqlalchemy.String, nullable=True)
    uf_name = sqlalchemy.Column(sqlalchemy.String, nullable=True)

    def __repr__(self):
        return "County(id=%s, county=%s, county_normalized=%s, uf=%s, uf_normalized=%s, uf_name=%s, uf_name_normalized=%s)"

# database

In [97]:
def make_operation(session):
    try:
        session.commit()
        session.flush()
    except Exception as e:
        session.rollback()
        print(e)
        raise
    finally:
        session.close()

def create_table_if_not_exists(table_name):
    if not sqlalchemy.inspect(engine).has_table(table_name, schema=cfg["database"]):
        Base.metadata.create_all(engine)

def create_datasp(info):
    return DataSP(seq=info["seq"],
                  modified=info["modified"], institution_code=info["institutionCode"],
                  collection_code=info["collectionCode"], catalog_number=info["catalogNumber"],
                  basis_of_record=info["basisOfRecord"], kingdom=info["kingdom"], phylum=info["phylum"],
                  classe=info["class"], order=info["order"], family=info["family"],
                  genus=info["genus"],
                  specific_epithet=info["specificEpithet"],
                  infraspecific_epithet=info["infraspecificEpithet"],
                  scientific_name=info["scientificName"],
                  scientific_name_authorship=info["scientificNameAuthorship"],
                  identified_by=info["identifiedBy"], year_identified=info["yearIdentified"],
                  month_identified=info["monthIdentified"], day_identified=info["dayIdentified"],
                  type_status=info["typeStatus"],
                  recorded_by=info["recordedBy"], record_number=info["recordNumber"],
                  field_number=info["fieldNumber"], year=info["year"], month=info["month"],
                  day=info["day"], event_time=info["eventTime"],
                  continent_ocean=info["continentOcean"], country=info["country"],
                  state_province=info["stateProvince"], county=info["county"], locality=info["locality"],
                  decimal_longitude=info["decimalLongitude"],
                  decimal_latitude=info["decimalLatitude"], verbatim_longitude=info["verbatimLongitude"],
                  verbatim_latitude=info["verbatimLatitude"],
                  coordinate_precision=info["coordinatePrecision"],
                  bounding_box=info["boundingBox"],
                  minimum_elevation_in_meters=info["minimumElevationInMeters"],
                  maximum_elevation_in_meters=info["maximumElevationInMeters"],
                  minimum_depth_in_meters=info["minimumDepthInMeters"],
                  maximum_depth_in_meters=info["maximumDepthInMeters"], sex=info["sex"],
                  preparation_type=info["preparationType"],
                  individual_count=info["individualCount"],
                  previous_catalog_number=info["previousCatalogNumber"],
                  relationship_type=info["relationshipType"],
                  related_catalog_item=info["relatedCatalogItem"],
                  occurrence_remarks=info["occurrenceRemarks"], barcode=info["barcode"],
                  imagecode=info["imagecode"], geo_flag=info["geoFlag"])

def create_county(json):
    uf, uf_name = get_uf(json)
    return County(id=get_id(json), county=get_county_name(json), uf=uf, uf_name=uf_name)

In [98]:
from sqlalchemy.sql.functions import ReturnTypeFromArgs

# CREATE EXTENSION unaccent;
class unaccent(ReturnTypeFromArgs):
    pass


def get_key(json, key):
    if key in json:
        return json[key]
    raise KeyError(f"key {key} not found")


def get_id(json):
    if "id" in json:
        return json["id"]
    raise KeyError(f"key id not found")


def get_county_name(json):
    if "nome" in json:
        return json["nome"]
    raise KeyError(f"key nome not found")


def get_uf(json):
    if "microrregiao" in json:
        if "mesorregiao" in json["microrregiao"]:
            if "UF" in json["microrregiao"]["mesorregiao"]:
                return json["microrregiao"]["mesorregiao"]["UF"]["sigla"], json["microrregiao"]["mesorregiao"]["UF"]["nome"]
            raise KeyError("key UF not found")
        raise KeyError("key mesorregiao not found")
    raise KeyError("key microrregiao not found")

C:\Users\xandao\AppData\Local\Temp\ipykernel_9288\3214596379.py:4: SAWarning: The GenericFunction 'unaccent' is already registered and is going to be overridden.
  class unaccent(ReturnTypeFromArgs):


In [99]:
create_table_if_not_exists("county")
create_table_if_not_exists("data")

2022-07-20 10:42:36,042 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-07-20 10:42:36,042 INFO sqlalchemy.engine.Engine [generated in 0.00266s] {'schema': 'herbario', 'name': 'county'}
2022-07-20 10:42:36,051 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:42:36,051 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-20 10:42:36,051 INFO sqlalchemy.engine.Engine [generated in 0.00087s] {'name': 'data'}
2022-07-20 10:42:36,062 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-07-20 10:42:36,062 INFO sqlalchemy.engine.Engine [cached since 0.005756s ago] {'name': 'county'}
2022-07-20 10:42:36,062 INFO sqlalchemy.engine.E

In [100]:
if session.query(County).count() == 0:
    try:
        response = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/municipios")
    except Exception as e:
        raise print(f"error: {e}")

    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)


if session.query(DataSP).count() == 0:

    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)


if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

2022-07-20 10:42:36,101 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-20 10:42:36,101 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT county.id AS county_id, county.county AS county_county, county.uf AS county_uf, county.uf_name AS county_uf_name 
FROM county) AS anon_1
2022-07-20 10:42:36,101 INFO sqlalchemy.engine.Engine [generated in 0.00129s] {}
2022-07-20 10:42:36,111 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scie

# Preprocess (dataframe)

In [101]:
def convert_header_to_snake_case(dataframe):
    return  {column_name: re.sub(r'(?<!^)(?=[A-Z])', '_', column_name).lower() for column_name in get_columns_dataframe(dataframe)}


def change_header(dataframe):
    dataframe.rename(columns=convert_header_to_snake_case(dataframe), inplace=True)


def get_columns_numeric(dataframe):
    list_of_columns_numeric = list([])
    for columns_dataframe in get_columns_dataframe(dataframe):
        for columns_table in get_columns_table(DataSP):
            if check_if_column_is_numeric(columns_dataframe, columns_table):
                list_of_columns_numeric.append(columns_dataframe)
    return list_of_columns_numeric


def get_columns_table(table):
    return table.__table__.columns


def get_columns_dataframe(dataframe):
    return list([*dataframe.columns])


def check_if_column_is_numeric(columns_dataframe, columns_table):
    return str(columns_dataframe) in str(columns_table) and ("int" in str(columns_table.type).lower() or "float" in str(columns_table.type).lower())


def replace_nan_to_null(dataframe):
    return dataframe.replace({numpy.nan: None})


def replace_values_not_numeric(dataframe):
    for column in list([*get_columns_numeric(dataframe)]):
        dataframe[column] = pandas.to_numeric(getattr(dataframe, column), errors='coerce').fillna(-1)
    return dataframe


def preprocess(dataframe):
    return replace_nan_to_null(replace_values_not_numeric(dataframe))

In [ ]:
data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(unaccent(County.county)).all()

In [115]:
def remove_set(data):
    return list([s for s, in data])

data_uf = remove_set(data_uf)
data_uf_name = remove_set(data_uf_name)
data_county = remove_set(data_county)
data_country = ["brasil", "brasil"]
list_of_all_data = [data_uf_name, data_county, data_country]

In [ ]:
def column_is_string_or_varchar_or_text(column):
    return "string" in str(column.type).lower() or "varchar" in str(column.type).lower() or "text" in str(column.type).lower()

list_seq = list([])
list_of_columns_valid = list([column.name.replace("data.", "") for column in get_columns_table(DataSP) if column_is_string_or_varchar_or_text(column)])

start = time.process_time()
for data in list_of_all_data:
    for d in data:
        for columns in list_of_columns_valid:
            separado = re.sub(r'\b[a-z]+\s*', '', d).strip().split() #
            if len(separado) > 0:
                q = session.query(DataSP.seq).filter(sqlalchemy.and_(*[sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{s}%") for s in separado])).all()
            else:
                q = session.query(DataSP.seq).filter(sqlalchemy.func.lower(unaccent(getattr(DataSP, columns))).ilike(f"%{d}%")).all()
            if len(q) > 0:
                list_seq.append({"seq": remove_set(q), "column": columns, "value_searched": d})
end = time.process_time()
elapsed_time = end-start
print(f"elapsed_time: {elapsed_time} seconds") 

In [117]:
pathlib.Path("result").mkdir(parents=True, exist_ok=True)
path_to_file = os.path.join("result", f"{datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')}.csv")
try:
    with open(path_to_file, "w") as file:
        file.write("seq; column; value_searched\n")
        for s in list_seq:
            file.write(f"{s['seq']}; {s['column']}; {s['value_searched']}\n")
        file.close()
        print(f"file {path_to_file} created")
except Exception as e:
    print(f"except: {e}")
    raise

file result\20-07-2022-10-46-14.csv created


In [106]:
session.close()
engine.dispose()

2022-07-20 10:42:37,861 INFO sqlalchemy.engine.Engine ROLLBACK
